<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Fundus-eye-disease-detection/Fixed_Custom_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Constants
IMG_SIZE = 224
CATEGORIES = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']
BATCH_SIZE = 32
EPOCHS = 25
DATA_DIR = '/content/drive/MyDrive/data set/dataset'

# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)
val_generator = train_datagen.flow_from_directory(
    DATA_DIR, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

# Compute class weights
y_train = train_generator.classes
class_weights_dict = dict(enumerate(compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train
)))

# Load pre-trained ResNet50
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze early layers
for layer in resnet_base.layers[:-5]:
    layer.trainable = False

# Feature extraction
x = GlobalAveragePooling2D()(resnet_base.output)
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(CATEGORIES), activation='softmax')(x)

# Model
model = Model(inputs=resnet_base.input, outputs=output)

# Compile with Adam optimizer
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=1e-6),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
]

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    class_weight=class_weights_dict,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

# Evaluate the model
val_loss, val_acc = model.evaluate(val_generator, steps=len(val_generator))
print(f"Validation Accuracy: {val_acc:.4f}")

# Get predictions
y_pred_probs = model.predict(val_generator, steps=len(val_generator))
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate accuracy
accuracy = accuracy_score(val_generator.classes, y_pred)
print(f"Accuracy on Validation Set: {accuracy:.4f}")

# Save the model
model.save('/content/custom_eye_disease_model.h5')
print("Model saved successfully.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 3384 images belonging to 4 classes.
Found 844 images belonging to 4 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 783s 7s/step - accuracy: 0.6806 - loss: 0.8691 - val_accuracy: 0.6848 - val_loss: 1.0449 - learning_rate: 0.0010
Epoch 2/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 94s 888ms/step - accuracy: 0.8129 - loss: 0.4899 - val_accuracy: 0.7133 - val_loss: 0.7916 - learning_rate: 0.0010
Epoch 3/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 141s 882ms/step - accuracy: 0.8463 - loss: 0.4316 - val_accuracy: 0.7346 - val_loss: 0.7363 - learning_rate: 0.0010
Epoch 4/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 142s 880ms/step - accuracy: 0.8636 - loss: 0.3803 - val_accuracy: 0.7761 - val_loss: 0.5555 - learning_rate: 0.0010
Epoch 5/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 142s 876ms/step - accuracy: 0.8612 - loss: 0.3592 - val_accuracy: 0.6588 - val_loss: 1.0044 - learning_rate: 0.0010
Epoch 6/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 93s 879ms/step - accuracy: 0.8787 - loss: 0.3333 - val_accuracy: 0.7346 - val_loss: 0.7144 - learning_rate: 0.0010
Epoch 7/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 94s 887ms/step - accuracy: 0.

Accuracy on Validation Set: 0.2666
Model saved successfully.
